<h1><strong> XGBoost 모델 훈련 및 학습 </strong></h1>
<br>

<hr>

<br>
<h2> 1. Introduction </h2>
 <p style="font-size:16px">개요 : 본 노트북은 프로젝트에서 사용될 파라미터 및 하이퍼파라미터 저장 과정을 기록한 리포트이다.
 </p>

<br>
<hr>
<br>
<h2> 2. Library Import </h2>


In [ ]:
import os
import numpy as np
import pandas as pd
import joblib
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error

import sys
sys.path.append("../src") 
import params

<hr>
<br>
<h2> 3. XGBoost 모델 훈련 및 저장 함수 </h2>

In [ ]:
def train_model(dataset_path: str):
    if not dataset_path or not os.path.exists(dataset_path):
        print("[INFO] 데이터셋이 존재하지 않습니다.")
        return

    print("[INFO] Started training...")
    df = pd.read_csv(dataset_path)
    X = df[['mean', 'std', 'high_freq_ratio']]
    y = df['target_k']
    
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
    print(f"[INFO] 학습 데이터: {len(X_train)}개, 검증 데이터: {len(X_val)}개")
    
    model = xgb.XGBRegressor(
        objective='reg:squarederror',
        n_estimators=1000,
        learning_rate=0.05,
        max_depth=5,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        n_jobs=-1,
        early_stopping_rounds=50
    )
    
    print("[INFO] Training data...")
    model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=False)
    
    preds = model.predict(X_val)
    mse = mean_squared_error(y_val, preds)
    print(f"[INFO] Successfully completed. Test Set MSE: {mse:.6f}")
    
    model_path = os.path.join(params.output_dir, "xgb_model.joblib")
    joblib.dump(model, model_path)
    print(f"[INFO] Saved the model : {model_path}")

    return model, X_train, y_train, X_val, y_val

<p style="font-size:16px">
   위 함수는 XGBoost 모델의 하이퍼파라미터를 지정하고 학습시킨다.

   이 모델의 특징(features)에는 각 패치별 평균 밝기, 분산, 고주파 비율 값들이 사용되고, 타겟(target)은 샤프닝강도 k값이다.

   부스팅 모델의 하이퍼파라미터에 objective를 MSE로 설정하여 오차율을 최소화하는 것을목표로 했고, 결정 트리의 개수(n_estimators)는 1000개, 학습률(learning_rate)은 0.05로 설정하여 조금씩 학습하는 대신많은 트리의개수로 꾸준하고 정밀하게 정답에 접근하도록 의도했다.
   
   또한 트리의 최대 깊이(max_depth)는 5로 제한하고, 데이터와 특성(column)의 샘플링 비율을 각각 0.8로 설정하여 과적합을 방지하며 일반화 성능을 높였다.

   특히 50회 동안 성능 개선이 없을 시 학습을 중단하는 조기 종료(early stopping_rounds) 조건을 추가했다. 데이터셋은 학습용과 검증용으로 8:2 비율로 분할하여 사용했으며, 학습 완료 후 검증 세트에 대한 평균 제곱 오차(MSE)를 통해 성능을 평가하고 최종 모델을 파일로 저장하도록 구현했다.
</p>

In [ ]:
def evaluate(model, X_train, y_train, X_val, y_val):

    X_train = X_train.drop(columns=['target_k'], errors='ignore')
    X_val = X_val.drop(columns=['target_k'], errors='ignore')

    y_pred_train = model.predict(X_train) # 학습한데이터 예측
    y_pred_val = model.predict(X_val) # 새로운 데이터 예측

    rmse_tr = np.sqrt(mean_squared_error(y_train, y_pred_train))    
    rmse_va = np.sqrt(mean_squared_error(y_val, y_pred_val))
    rmae_tr = np.sqrt(mean_absolute_error(y_train, y_pred_train))
    rmae_va = np.sqrt(mean_absolute_error(y_val, y_pred_val))
    
    print("========== Evaluate ==========")
    print("RMSE of Train Set : ", rmse_tr)
    print("RMSE of Test Set : ", rmse_va)
    print()
    print("RMAE of Train Set : ", rmae_tr)
    print("RMAE of Test Set : ", rmae_va)

<p style="font-size:16px">
위 함수는 학습된 모델의 성능을 검증하고, 과적합 여부를 판단하기 위해 지표를 계산하여 출력하는 함수이다.

평균 제곱 오차의 제곱근, 평균 절대 오차의 제곱근을 사용했다.
k값을 정확히 예측하는 것에 목표가 아닌 영역의 경향성을 예측하면 되기 때문에 결정계수(R2)나 다른 추가 지표는 사용하지 않았다.
</p>

<hr>

<h2> 4. 결론 </h2>
<p style="font-size:16px">
- 과적합을 방지하기 위해 <code>learning_rate</code>와 <code>max_depth</code>, 샘플링 등의 하이퍼파라미터를 조정하였으며, 조기 종료 기법을 적용하여 성능을 향상시켰다.<br>
- 평가지표로 RMSE, RMAE, R2로 과적합, 과격한 오차, 학습률을 비교했다.<br>
    <br><br>

이제 학습된 XGBoost 모델을 로드하여, 실제 저화질 이미지에 적응형 언샤프 마스킹(Adaptive Unsharp Masking)을 적용해보고 화질 개선 효과를 시각적으로 검증하는 시연 단계를 진행할 예정이다.

    -> notebooks/inference.ipynb
</p>